# Order Management Challenge

**Objective:** 
Given a guest $check$, the guest decide to $change$ one of the items for another. The function should return a new modified check.

* Input: **List** check, **String** new_order
* Output: **List** new_check


**Restrinctions**
* The function works only with english words.
* The check is a list, items are composed by strings, each item have a int number, 0..N adjetives and 1 noun.
* The intent of the $new\_order$ is always a change.
* The $new\_order$ always reference to an existing item in the $check$.
* The item reference in $new\_order$ can be singular, plural or synonym.



**Examples**:
- $check$: 
    - ["2 large pepperoni pizzas", "3 sugar free sodas"]

- $new\_order$:
    - "Make one soda reguar" **singular version**
    - "Make one of the drinks a regular" **synonym**
    
- $new\_check$:
    - ["2 large pepperoni pizzas", "2 sugar free sodas", "1 regular soda"]
    
    
    
    
**Goal**:

We are given:
- A $check$ array with items where the amount is indicated with a integer
- The $new\_order$ is always a $change$ intent from an existing item to another, regardless of being a singular, plural or synonym of the item

Our goal and logic:
    1. Detect the original $check$ items and amounts
    2. Detect the new_order item and amount
    3. Detect the similarity between the new_order and the original check to known where we need to execute the change
    4. Execute the change:
        - Decrease the amount of existing item, if amount is 0 it should be removed
        - Add the new item to the list
        
As additional logic we will contemplate the cases when no numbers are specified, this can have different expressions:
- Singular: "Change a water for juice"        --> Here we should remove 1 water and add 1 juice
- Plural:   "Change the waters for juices"    --> Here we should remove all waters and add the same amount of juices
    



**SpaCy Features**

* Tokenization: is the task of splitting a text into meaningful segments, called tokens.

* Tokens similarity: similarity is determined by comparing word vectors or “word embeddings”, multi-dimensional meaning representations of a word. Word vectors can be generated using an algorithm like word2vec.

* en_vectors_web_lg dataset: this dataset provides a large vocabulary with more vectors, including over 1 million unique vectors. In other words we benefit from this having larger vocabulary information.

## Libs Imports
- spacy: NLP library
- displacy: spacy data displayer, just for this notebook
- en_core_web_lg: English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Used for word tokenization. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.
- lemmatizer: spacy lib to compute word lemmas
- Counter: collections helper function
- tabulate, display, HTML: table display libs

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_lg
nlp = en_core_web_lg.load()
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
from spacy.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
from tabulate import tabulate
from IPython.core.display import display, HTML
from check_change_module import *

### How to analyze words?

**SpaCy**

To solve this exercise we can use [SpaCy](https://spacy.io/), a NLP python library which provides features as tokenization, med entity recognition and support for many languages.

Using the spacy displayer we can see how it works, it decomposes sentences doing segmentation and word tagging. With this we can detect how the words are related and the role of each one.

**Token analysis:**
* Text: The original word text.
* Lemma: The base form of the word.
* POS: The simple part-of-speech tag.
* Tag: The detailed part-of-speech tag.
* Dep: Syntactic dependency, i.e. the relation between tokens.
* Shape: The word shape – capitalization, punctuation, digits.
* Alpha: Is the token an alpha character?
* Stop: Is the token part of a stop list, i.e. the most common words of the language?

In [2]:
headers = "TEXT,LEMMA,POS,TAG,DEP,SHAPE,ALPHA,STOP".split(',')
table = []
doc = nlp('3 sugar free sodas')
for token in doc:
    table.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])

print("Tokens info:")
display(HTML(tabulate(table, headers, tablefmt="html")))

print("Words relation:")
displacy.render(doc, jupyter=True, style='dep')

Tokens info:


TEXT,LEMMA,POS,TAG,DEP,SHAPE,ALPHA,STOP
3,3,NUM,CD,nummod,d,False,False
sugar,sugar,NOUN,NN,npadvmod,xxxx,True,False
free,free,ADJ,JJ,amod,xxxx,True,False
sodas,soda,NOUN,NNS,ROOT,xxxx,True,False


Words relation:


## Sentence class

This class process all the components and behaviour of a sentence.

In [3]:
sentence_1 = Sentence('2 large pepperoni pizzas')
sentence_2 = Sentence('Make one of the pizzas small')
sentence_3 = Sentence('Change the pizzas for pastas')

sts = [sentence_1, sentence_2, sentence_3]

for st in sts:
    print(f"Sentence: {st.sentence}")
    print(f"Main Noun: {st.noun}")
    print(f"Adjetive: {st.adjetive}")
    print(f"Number: {st.number}")
    print(f"Adposition: {st.adposition}")
    print(f"Is plural?: {st.is_plural(sentence_1.noun)}")
    print(" ")

Sentence: 2 large pepperoni pizzas
Main Noun: pizzas
Adjetive: large pepperoni
Number: 2
Adposition: 
Is plural?: True
 
Sentence: Make one of the pizzas small
Main Noun: pizzas
Adjetive: small
Number: 1
Adposition: of
Is plural?: True
 
Sentence: Change the pizzas for pastas
Main Noun: pastas
Adjetive: 
Number: 0
Adposition: for
Is plural?: True
 


## Order manager class

This class is in charge of managing the order, comparing similarity between items and making the changes.

In [4]:
# Init manager and check result
check = ['2 large pepperoni pizzas', '3 sugar free sodas']
new_order = 'Make one of the pizzas small'
order_manager = OrderManager(check, new_order) 

print(order_manager.new_check)

['1 large pepperoni pizzas', '3 sugar free sodas', '1 small pizzas']


## Main function

Receives parameters, call the manager and return new check

In [5]:
def change_check(check, new_order):
    """Main function. Receives the check and new order, returns the new check.
        arguments: 
          string array -- the array with the order items
          string       -- the change requeriment 
        return: 
          string array -- the new check
    """
    order_manager = OrderManager(check, new_order)   
    new_check = order_manager.new_check

    return new_check

## Testing

Create an example check, load some test new orders and validate results

In [6]:
# %load tests.py
from check_change_module import *

check = ['2 large pepperoni pizzas', '3 sugar free sodas']
test_new_orders = [
    'Make one of the pizzas small',
    'Make two of the drinks a regular',
    'Make 3 sodas regular',
    'Change a soda for wine',
    'Change the pizzas for pastas',
    'Change the sodas with cocktails'
]

valid_new_checks = [
    ['1 large pepperoni pizzas', '3 sugar free sodas', '1 small pizzas'],
    ['2 large pepperoni pizzas', '1 sugar free sodas', '2 regular drinks'],
    ['2 large pepperoni pizzas', '3 regular sodas'],
    ['2 large pepperoni pizzas', '2 sugar free sodas', '1  wine'],
    ['3 sugar free sodas', '2  pastas'],
    ['2 large pepperoni pizzas', '3  cocktails']
]

for key, new_order in enumerate(test_new_orders):
    new_check = change_check(check, new_order)
    assert new_check == valid_new_checks[key], f"Error in key {key}. The generated new check is not valid: {new_check}"